In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

## Setting up the basics

In [7]:
mnist = input_data.read_data_sets('./inputs/mnist')
# Resetting default graph, starting from scratch
tf.reset_default_graph()

batch_size = 64
n_noise = 64
learning_rate=0.00015

real_images = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28], name='X')
noise = tf.placeholder(dtype=tf.float32, shape=[None, n_noise])

# The keep_prob variable will be used by our dropout layers, which we introduce for more stable learning outcome
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')

# Leaky Relu activation
# https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29#Potential_problems
def lrelu(x):
    return tf.maximum(x, tf.multiply(x, 0.2))

# Binary cross entropy for descriminators
def binary_cross_entropy(x, z):
    eps = 1e-12
    return (-(x * tf.log(z + eps) + (1. - x) * tf.log(1. - z + eps)))

Extracting ./inputs/mnist/train-images-idx3-ubyte.gz
Extracting ./inputs/mnist/train-labels-idx1-ubyte.gz
Extracting ./inputs/mnist/t10k-images-idx3-ubyte.gz
Extracting ./inputs/mnist/t10k-labels-idx1-ubyte.gz


# The descriminator

In [8]:
# It takes either real or fake MNIST image 28 x 28 in grayscale
# we use a sigmoid to make sure our output can be interpreted 
# as the probability the input image is a real MNIST character.
def discriminator(X_in, reuse=None, keep_prob=keep_prob):
    activation=lrelu
    with tf.variable_scope('disc', reuse=reuse):
        print('checking shape')
        print(X_in.shape)
        x = tf.reshape(X_in, shape=[-1, 28, 28, 1])
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, units=128, activation=activation)
        x = tf.layers.dense(x, units=1, activation=tf.nn.sigmoid)
        return x

# The generator

In [9]:
# z => noise
def generator(z, keep_prob=keep_prob, is_training=is_training):
    activation = lrelu
    momentum = 0.99
    with tf.variable_scope('gen', reuse=None):
        # Simply reassigning noise to x to begin with
        x = z
        d1 = 4
        d2 = 1
        x = tf.layers.dense(x, units=d1 * d1 * d2, activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        # https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.reshape(x, shape=[-1, d1, d1, d2])
        x = tf.image.resize_images(x, size=[7, 7])
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        x = tf.layers.dropout(x, keep_prob)
        x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
        x = tf.layers.conv2d_transpose(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        return x

# Loss functions and optimizers

In [10]:
g = generator(noise, keep_prob, is_training)
d_real = discriminator(real_images)
d_fake = discriminator(g, reuse=True)

print(d_real)
vars_g = [var for var in tf.trainable_variables() if 'gen' in var.name]
vars_d = [var for var in tf.trainable_variables() if 'disc' in var.name]

# Applying regularizers
d_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_d)
g_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_g)

loss_d_real = binary_cross_entropy(tf.ones_like(d_real), d_real)
loss_d_fake = binary_cross_entropy(tf.zeros_like(d_fake), d_fake)
loss_g = tf.reduce_mean(binary_cross_entropy(tf.ones_like(d_fake), d_fake))
loss_d = tf.reduce_mean(0.5 * (loss_d_real + loss_d_fake))

optimizer_d = tf.train.AdamOptimizer(learning_rate).minimize(loss_d, var_list=vars_d)
optimizer_g = tf.train.AdamOptimizer(learning_rate).minimize(loss_g, var_list=vars_g)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

checking shape
(?, 28, 28)
checking shape
(?, 28, 28, 64)
Tensor("disc/dense_2/Sigmoid:0", shape=(?, 1), dtype=float32)
